In [1]:
import pandas as pd, numpy as np
import glob

In [2]:
def fileListLoad(path : str) :
    lst = []
    fileList = glob.glob(path)

    for f in fileList :
        df = pd.read_sas(f)
        lst.append(df)

    return lst

In [3]:
df_main = fileListLoad('Data/all/*.sas7bdat')

In [4]:
df_2019 = df_main[0].copy()
df_2020 = df_main[1].copy()
df_2021 = df_main[2].copy()

In [6]:
df_sup = fileListLoad('Data/sup/*.sas7bdat')

In [7]:
df_2019_sup = df_sup[0].copy()
df_2020_sup = df_sup[1].copy()
df_2021_sup = df_sup[2].copy()

In [8]:
df_2021_sup.columns

Index(['mod_d', 'ID', 'ID_fam', 'year', 'region', 'town_t', 'apt_t', 'psu',
       'sex', 'age', 'age_month', 'incm', 'ho_incm', 'incm5', 'ho_incm5',
       'edu', 'occp', 'wt_hs', 'wt_itvex', 'wt_oe', 'wt_ntr', 'wt_tot',
       'wt_oent', 'kstrata', 'LS_SUP', 'LS_KIND', 'LS_CODE', 'LS_NAME',
       'LS_CNAME', 'LS_DUR', 'LS_FQ', 'LS_FQ_U', 'LS_SUP_1D', 'LS_AM',
       'LS_AM_U', 'LS_AM_OU', 'LS_CA', 'LS_PHOS', 'LS_FE', 'LS_VA', 'LS_B1',
       'LS_B2', 'LS_NIAC', 'LS_VITC'],
      dtype='object')

In [10]:
df_2021.iloc[0,:]

mod_d        b'2023.12.07.'
ID            b'A901215301'
ID_fam          b'A9012153'
year                 2021.0
region                  1.0
                  ...      
HEI_SUGAR          9.676108
HEI_CHO            4.464322
HEI_FAT                 5.0
HEI_EN             0.901755
HEI               45.784216
Name: 0, Length: 864, dtype: object

In [9]:
df_2021_sup.iloc[0,:]

mod_d                                           b'2023.12.07.'
ID                                               b'A901215301'
ID_fam                                             b'A9012153'
year                                                    2021.0
region                                                     1.0
town_t                                                     1.0
apt_t                                                      2.0
psu                                                    b'A901'
sex                                                        1.0
age                                                       61.0
age_month                                                  NaN
incm                                                       3.0
ho_incm                                                    3.0
incm5                                                      4.0
ho_incm5                                                   4.0
edu                                                    

영양제 column은 

In [44]:
df_2021.shape == df_2021_sup.shape

False

`LS_KIND`가 다른 등 이유로 인해, 해당 dataset의 key에 해당하는 `ID`가 영양 보충제 부분에서 2개 이상 로우에 존재하는 등 문제가 발생하는 자료이다. 영양 보충제 섭취에 관한 자료를 main dataframe에 join하기 위해 정보를 key별로 요약해야 했는데, 각 key별로 하나의 row로 여러 보충제 정보를 얻기 위해 다음의 절차를 따랐다.

* `LS_SUP` : 식이보충제 복용 여부 더미이므로 보존한다. ID별로 다 더하고 더한 개수로 나누면 될 것이다.
* `LS_KIND` : 하나의 row로 줄일 것이므로 제거한다. 
* `LS_CODE`, `LS_NAME`, `LS_AM_OU`, `LS_AM_U`, `LS_AM`, `LS_SUP_ID`, `LS_FQ_U` : 식이보충제 자체에 대한 정보를 담고 있는 값이고 사용하기 어려우므로 드랍한다.

* `LS_DUR`, `LS_FQ` : maximum의 값만을 보존한다.

* `LS_CA`, `LS_PHOS`, `LS_FE`, `LS_VA`, `LS_B1`, `LS_B2`, `LS_NIAC`, `LS_VITC` : 식이보충제 제품별로 영양분이 표시되어 있는 칼럼이므로, 개인별로 해당 칼럼을 모두 더해 사용한다.

In [11]:
sumlist = ['LS_CA', 'LS_PHOS', 'LS_FE', 'LS_VA', 'LS_B1', 'LS_B2', 'LS_NIAC', 'LS_VITC']
droplist = ['mod_d', 'LS_CODE', 'LS_NAME', 'LS_AM_OU', 'LS_AM_U', 'LS_AM', 'LS_SUP_1D', 'LS_FQ_U']

In [13]:
def toOneRow(df, slist, dlist):
    df.drop(columns = dlist, inplace=True)
    gdf = df.groupby('ID').agg({
        **{col: 'sum' for col in slist},
        'LS_SUP': 'mean',
        'LS_DUR': 'max',
        'LS_PHOS': 'max'
    }).reset_index()
    return gdf

In [16]:
df_2021_fsup = toOneRow(df_2021_sup, sumlist, droplist)

KeyError: "['mod_d', 'LS_CODE', 'LS_NAME', 'LS_AM_OU', 'LS_AM_U', 'LS_AM', 'LS_SUP_1D', 'LS_FQ_U'] not found in axis"

In [ ]:
df_2021_fsup

,ID,LS_CA,LS_PHOS,LS_FE,LS_VA,LS_B1,LS_B2,LS_NIAC,LS_VITC,LS_SUP,LS_DUR
0,b'A901215301',0.00,0.000,0.000000,0.0,0.0,0.0,0.0,1000.0,1.0,5.0
1,b'A901225501',500.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,b'A901234001',0.00,0.000,0.000000,0.0,0.0,1.4,0.0,1000.0,1.0,5.0
3,b'A901235701',0.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,5.0
4,b'A901237402',300.00,20.000,8.000000,1050.0,1.1,1.1,14.0,100.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
3631,b'R904339403',0.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,2.0
3632,b'R904353001',38.61,75.465,1.219725,0.0,0.0,0.0,0.0,0.0,1.0,5.0
3633,b'R904353002',0.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,5.0
3634,b'R904353003',500.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,5.0


In [17]:
df_2021f = pd.merge(df_2021, df_2021_fsup, how = 'left')

In [18]:
df_2021f.drop(columns = 'mod_d', inplace = True)

In [19]:
df_2021f['LS_SUP'].fillna(0.0, inplace = True)
df_2021f

C:\Users\Na\AppData\Local\Temp\ipykernel_34272\2445172108.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2021f['LS_SUP'].fillna(0.0, inplace = True)


,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,age_month,...,LS_CA,LS_PHOS,LS_FE,LS_VA,LS_B1,LS_B2,LS_NIAC,LS_VITC,LS_SUP,LS_DUR
0,b'A901215301',b'A9012153',2021.0,1.0,1.0,2.0,b'A901',1.0,61.0,NaN,...,0.00,0.000,0.000000,0.0,0.0,0.0,0.0,1000.0,1.0,5.0
1,b'A901215302',b'A9012153',2021.0,1.0,1.0,2.0,b'A901',1.0,57.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,b'A901225501',b'A9012255',2021.0,1.0,1.0,2.0,b'A901',2.0,39.0,NaN,...,500.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,b'A901225502',b'A9012255',2021.0,1.0,1.0,2.0,b'A901',2.0,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,b'A901227201',b'A9012272',2021.0,1.0,1.0,2.0,b'A901',1.0,70.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7085,b'R904351302',b'R9043513',2021.0,8.0,2.0,2.0,b'R904',1.0,25.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
7086,b'R904353001',b'R9043530',2021.0,8.0,2.0,2.0,b'R904',1.0,45.0,NaN,...,38.61,75.465,1.219725,0.0,0.0,0.0,0.0,0.0,1.0,5.0
7087,b'R904353002',b'R9043530',2021.0,8.0,2.0,2.0,b'R904',2.0,43.0,NaN,...,0.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,5.0
7088,b'R904353003',b'R9043530',2021.0,8.0,2.0,2.0,b'R904',1.0,8.0,NaN,...,500.00,0.000,0.000000,0.0,0.0,0.0,0.0,0.0,1.0,5.0


In [62]:
df_2021_sup['LS_CNAME'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 8167 entries, 0 to 8166
Series name: LS_CNAME
Non-Null Count  Dtype 
--------------  ----- 
8167 non-null   object
dtypes: object(1)
memory usage: 63.9+ KB


In [20]:
df_2020_fsup = toOneRow(df_2020_sup, sumlist, droplist)

df_2020f = pd.merge(df_2020, df_2020_fsup, how = 'left')

df_2020f.drop(columns = 'mod_d', inplace = True)
df_2020f['LS_SUP'].fillna(0.0, inplace = True)

df_2020f

C:\Users\Na\AppData\Local\Temp\ipykernel_34272\1525755808.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2020f['LS_SUP'].fillna(0.0, inplace = True)


,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,age_month,...,LS_CA,LS_PHOS,LS_FE,LS_VA,LS_B1,LS_B2,LS_NIAC,LS_VITC,LS_SUP,LS_DUR
0,b'A801169401',b'A8011694',2020.0,1.0,1.0,2.0,b'A801',1.0,39.0,NaN,...,4.205000,0.000000,0.000000,0.0,28.441935,50.000000,50.000000,30.000000,1.0,2.0
1,b'A801169402',b'A8011694',2020.0,1.0,1.0,2.0,b'A801',2.0,39.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,b'A801169403',b'A8011694',2020.0,1.0,1.0,2.0,b'A801',1.0,10.0,NaN,...,0.000000,0.000000,0.000000,135.0,0.300000,0.400000,5.000000,20.000000,1.0,3.0
3,b'A801169404',b'A8011694',2020.0,1.0,1.0,2.0,b'A801',1.0,7.0,NaN,...,0.000000,0.000000,0.000000,135.0,0.300000,0.400000,5.000000,20.000000,1.0,3.0
4,b'A801169405',b'A8011694',2020.0,1.0,1.0,2.0,b'A801',1.0,4.0,58.0,...,0.000000,0.000000,0.000000,135.0,0.300000,0.400000,5.000000,20.000000,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7354,b'R804366601',b'R8043666',2020.0,8.0,1.0,2.0,b'R804',2.0,61.0,NaN,...,133.346948,8.474178,6.248826,0.0,17.021362,17.017136,20.039906,133.046948,1.0,3.0
7355,b'R804368301',b'R8043683',2020.0,8.0,1.0,2.0,b'R804',2.0,51.0,NaN,...,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,3.0
7356,b'R804368302',b'R8043683',2020.0,8.0,1.0,2.0,b'R804',2.0,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
7357,b'R804371701',b'R8043717',2020.0,8.0,1.0,2.0,b'R804',1.0,65.0,NaN,...,0.000000,0.292032,0.000000,0.0,33.290561,2.107519,0.000000,70.000000,1.0,2.0


In [21]:
df_2019_fsup = toOneRow(df_2019_sup, sumlist, droplist)

df_2019f = pd.merge(df_2019, df_2019_fsup, how = 'left')

df_2019f.drop(columns = 'mod_d', inplace = True)
df_2019f['LS_SUP'].fillna(0.0, inplace = True)

df_2019f

C:\Users\Na\AppData\Local\Temp\ipykernel_34272\61493291.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_2019f['LS_SUP'].fillna(0.0, inplace = True)


,ID,ID_fam,year,region,town_t,apt_t,psu,sex,age,age_month,...,LS_CA,LS_PHOS,LS_FE,LS_VA,LS_B1,LS_B2,LS_NIAC,LS_VITC,LS_SUP,LS_DUR
0,b'A751215301',b'A7512153',2019.0,1.0,1.0,1.0,b'A751',1.0,61.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,b'A751215303',b'A7512153',2019.0,1.0,1.0,1.0,b'A751',1.0,28.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,b'A751220401',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',1.0,53.0,NaN,...,0.00,0.00,0.000,264.000000,0.4000,0.5400,0.000,37.0,1.0,4.0
3,b'A751220402',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',2.0,50.0,NaN,...,75.00,0.00,0.000,1164.000000,8.4000,8.5400,24.000,92.0,1.0,2.0
4,b'A751220403',b'A7512204',2019.0,1.0,1.0,1.0,b'A751',1.0,16.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8105,b'R754381902',b'R7543819',2019.0,8.0,1.0,2.0,b'R754',2.0,48.0,NaN,...,0.18,0.16,0.027,0.030833,0.0014,0.0083,0.017,0.9,1.0,5.0
8106,b'R754393801',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',1.0,47.0,NaN,...,0.00,0.00,0.000,0.000000,0.0000,0.0000,0.000,1000.0,1.0,2.0
8107,b'R754393802',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',2.0,43.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
8108,b'R754393803',b'R7543938',2019.0,8.0,1.0,2.0,b'R754',2.0,16.0,NaN,...,0.00,0.00,0.000,0.000000,0.0000,0.0000,0.000,1000.0,1.0,2.0
